In [1]:
import os
# Set spark environments
os.environ['PYSPARK_PYTHON'] = r'C:\Users\room102sys2\AppData\Local\Programs\Python\Python39'
#os.environ['PYSPARK_DRIVER_PYTHON'] = r'C:\Users\room102sys2\AppData\Local\Programs\Python\Python39\Scripts\jupyter'
#os.environ['']

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
import findspark
findspark.init('C:\Program Files\Spark')

In [4]:
sp = SparkSession.builder.appName("Data Sources").getOrCreate()
sc = sp.sparkContext

Spark has Six core data sources and hundreds of external data sources written by the community. The ability to read and write from all different kinds of data sources and for the community to create its own contributions is one of Sparks greatest strength.
Following are Spark's core data sources:
- CSV
- JSON
- Parquet
- ORC
- JDBC/ODBC connections
- Plain-text files

And Spark has numerous community-created data sources. Few of them are mentioned here:

- Cassandra
- HBase
- MongoDB
- AWS Redshift
- XML
- And many others

## Read API Structure
The core structure for reading data is as follows:

`DataFrameReader.format(...).option("key","value").schema(...).load()`

- `format` is optional because by default Spark will Use the Parquet format.
- `option` allows us to key-value configurations to parameterize how we read data.
- `schema` is optional if the data source provides a schema or if you intended to use schema inference.

## Basics of Reading Data
The basic thing for reading data in Spark is the `DataFrameReader`. We access this through the `SparkSession` via the read attribute:

`spark.read`

After we have a DataFrame reader, we specify several values:

- The `format`
- The `schema`
- The *read mode*
- A series of *`options`*


- The format options, and schema each return a `DataFrameReader` that can undergo further tranformations and are all optional, except for one option.
- Each data source has a specific set of options that determine how the data is read into spark.
- At a minimum, we must supply the `DataFrameReader` a path to from which to read.

Here is the example of the overall layout:
```
spark.read.format("csv")
  .option("mode","FAILFAST")
  .option("path","path/to/file(s)")
  .schema(someSchema)
  .load()
  ```

### Read modes
- Reading data from an external source naturally entails encountring malformed data, especially when working with only semi-structured data sources.
- Read modes specify what will happen when Spark does come across malformed records.


| Read mode       | Description                                                                                             |  
|-----------------|---------------------------------------------------------------------------------------------------------|
| `permissive`    | Sets all fields to `null` when it encounters a corrupted record and places all corrupted records in a   |   
|                  |string column `called_corrupt_record`                                                                   | 
| `dropMalformed` | Drops the row that contains malformed records                                                           |   
| `failFast`      | Fails immediatley upon encountring malformed records                                                    |   |-----------------|---------------------------------------------------------------------------------------------------------|

The default is `permissive`

### Write API Structure
The core structure for writing data is as follows:
`DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).save()`

- `format` is optional because by default, spark will use the parquet format. 
- `option` allows us to configure how to write out our given data.
- `PartitionBy`, `bucketBy`, and `sortBy` work only for file-based data sources; We can use them to control the specific layout of files at the destinations.

## Basics of Writing Data

- For writing data we use `DataFrameWriter`. 
- After we have a `DataFrameWriter`, we specify three values: the `format`, a series of `options`, and the `save` mode. 
- At a minimum, we must supply a path.

Here is the syntax for writing data:
```
dataframe.write.format("csv")
    .option("mode","OVERWRITE")
    .option("dateformat","yyyy-MM-dd")
    .option("path","path/to/file(s)")
    .save()
    ```

### Save modes
Save modes specify what will happen if Spark finds data at the specified location. Following table lists the save modes:

| Save mode       | Description                                                                                  |
|-----------------|----------------------------------------------------------------------------------------------|
| `append`        | Appends the output files to the list of files that already exist at that location.           |
| `overwrite`     | will completely overwrite any data that already exists there.                                |
| `errorIfExists` | Throws an error and fails the write if data or files already exist at the specified location |
| `ignore`        | If data or files exist at the location, do nothing with the current DataFrame                |

The default is `errorIfExists`.This means that if spark finds data at the location to which we are writing, it will fail the write immediatley.

## CSV Files
- CSV stands for comma-separated values.
### CSV Options

| read/write| key                  | Potential values           | Default  | Description                                   | 
|-----------|----------------------|----------------------------|----------|-----------------------------------------------|
| Both      | compression or codec |None,uncompressed,          |          |Declares what compression codec Spark          |
|           |                      |bzip2, deflate,             | none     |                                               |
| Both      | dateFormat           |Any string orcharacter that |          |                                               |
|           |                      |conforms to Java’s          |          |                                               |
|           |                      |SimpleDataFormat            |yyyy-MM-dd| 
| `errorIfExists` | Throws an error and fails the write if data or files already exist at the specified location |
| `ignore`        | If data or files exist at the location, do nothing with the current DataFrame                |

### Reading CSV Files

- To read CSV file first we need to create a `DataFrameReader`. Here is the syntax:

`spark.read.format("csv")`

Here is the expanded syntax:

`spark.read.format("csv")
    .option("header","true")
    .option("mode","FAILFAST")
    .option("inferschema","true")
    .load("some/path/to/file.csv")`
    
    

In [5]:
from pyspark.sql.types import StructField, StructType, StringType, LongType
myManualSchema = StructType([
StructField("DEST_COUNTRY_NAME", StringType(), True),
StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
StructField("count", LongType(), False)
])

sp.read.format("csv").option("header", "true").option("mode", "FAILFAST").schema(myManualSchema).load("F:/proj_pyspark/data/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



If you give incorrect data type of spark schema. The problem mainfest itself when Spark actually reads the data. As soon as we start our Spark job, it will immediatley fail due to the data not confirming to the specified schema.

In [6]:
myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", LongType(), True),
    StructField("ORIGIN_COUNTRY_NAME", LongType(), True),
    StructField("count", LongType(), False) ])

In [7]:
sp.read.format("csv").option("header", "true").option("mode", "FAILFAST").schema(myManualSchema).load("F:/proj_pyspark/data/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv").take(5)

Py4JJavaError: An error occurred while calling o44.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (room102csesys9 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:400)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at org.apache.spark.sql.execution.da

- In genral Spark will fail only at job execution time rather than DataFrame dafinition time-----this is called __lazy evaluation__.

### Writing CSV Files
Lets see the simple example for writing data after reading:


In [8]:
csvFile = sp.read.format("csv")\
.option("header", "true")\
.option("mode", "FAILFAST")\
.option("inferSchema", "true")\
.load("F:/proj_pyspark/data/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv")

In [9]:
csvFile.write.format("csv").mode("overwrite").option("sep", "\t").save("my-tsv-file.tsv")

Py4JJavaError: An error occurred while calling o55.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at or

When we list the destination directory, you can see that `my-tsv-file` is actually a folder with numerous
files within it:

## JSON Files
- In Spark, when we refer to JSON files, we refer to linedelimited JSON files.
- The line-delimited versus multiline trade-off is controlled by a single option: multiLine.
- Line-delimited JSON is actually a much more stable format because it allows you to append to a file with a new record (rather   than having to read in an entire file and then write it out), which is what we recommend that you use.
- The line-delimited versus multiline trade-off is controlled by a single option: multiLine. When you set this option to true,   you can read an entire file as one json object and Spark will go through the work of parsing that into a DataFrame.
- Line-delimited JSON is actually a much more stable format because it allows you to append to a file with a new record (rather   than having to read in an entire file and then write it out), which is what we recommend that you use.
- Another key reason for the popularity of line-delimited JSON is because JSON objects have structure, and JavaScript (on which
  JSON is based) has at least basic types.

### JSON Options


### Reading JSON Files
Lets see an example of reading JSON file and its options.

In [ ]:
# in Python
sp.read.format("json").option("mode", "FAILFAST").option("inferSchema", "true").load("./data/Spark-The-Definitive-Guide/data/flight-data/json/2010-summary.json").show(5)

### Writing JSON Files
- Writing JSON files is just as simple as reading them, and, as you might expect, the data source does not matter.


In [ ]:
csvFile.write.format("json").mode("overwrite").save("/tmp/my-json-file.json")

## Parquet Files
- Parquet is an open source column-oriented data store that provides a variety of storage optimizations, especially for analytics workloads.
- It provides columnar compression, which saves storage space and allows for reading individual columns instead of entire files.
- It is a file format that works exceptionally well with Apache Spark and is in fact the default file format.
- Another advantage of Parquet is that it supports complex types. This means that if your column is an array (which would fail with a CSV file, for example), map, or struct, you’ll still be able to read and write that file without issue.

Syntax:

`spark.read.format("parquet")`

### Reading Parquet Files
- Parquet has very few options because it enforces its own schema when storing data.
- We can set the schema if we have strict requirements for what our DataFrame should look like.


In [ ]:
sp.read.format("parquet").load("./data/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet").show(5)

### Parquet options
 
| Read/Write | Key                 | Potential Values   | Default                     |   Description                         |
|------------|---------------------|--------------------|-----------------------------|---------------------------------------|
|Write       |compression or codec |None, uncompressed, | None                        | Declares what compression codec       |
|            |                     |bzip2, deflate,     |                             |Spark should use to read or write the  |
|            |                     |gzip, lz4, or snappy|                             |file.                                  |
|Read        |mergeSchema          |true, false         |Value of the configuration   |You can incrementally add columns to   |
|            |                     |                    |spark.sql.parquet.mergeSchema|newly written Parquet files in the same|
|            |                     |                    |                             |table/folder. Use this option to enable|
|            |                     |                    |                             | disable this feature.                 |


### Writing Parquet Files
Writing Parquet is as easy as reading it. 

In [ ]:
# in Python
csvFile.write.format("parquet").mode("overwrite").save("/tmp/my-parquet-file.parquet")

## ORC Files
- ORC is a self-describing, type-aware columnar file format designed for Hadoop workloads. It is optimized for large streaming reads, but with integrated support for finding required rows quickly.
- ORC actually has no options for reading in data because Spark understands the file format quite well.
- The fundamental difference is that Parquet is further optimized for use with Spark, whereas ORC is further optimized for Hive.

### Reading Orc Files

In [ ]:
# in Python
sp.read.format("orc").load("./data/Spark-The-Definitive-Guide/data/flight-data/orc/2010-summary.orc").show(5)

### Writing Orc Files


In [ ]:
csvFile.write.format("orc").mode("overwrite").save("/tmp/my-json-file.orc")

## SQL Databases

- SQL datasources are one of the more powerful connectors because there are a variety of systems to which you can connect (as long as that system speaks SQL).
- To read and write from these databases, you need to do two things: include the Java Database Connectivity (JDBC) driver for you particular database on the spark classpath, and provide the proper JAR for the driver itself.

| Property Name | Meaning |  |
|---|---|---|
| url | The JDBC URL to which to connect. The source-specific connection properties can be specified in<br>the URL; for example, jdbc:postgresql://localhost/test?user=fred&password=secret. |  |
| dbtable | The JDBC table to read. Note that anything that is valid in a FROM clause of a SQL query can be<br>used. For example, instead of a full table you could also use a subquery in parentheses. |  |
| driver | The class name of the JDBC driver to use to connect to this URL. |  |
| partitionColumn,<br>lowerBound, upperBound | If any one of these options is specified, then all others must be set as well. In addition,<br>numPartitions must be specified. These properties describe how to partition the table when reading<br>in parallel from multiple workers. partitionColumn must be a numeric column from the table in<br>question. Notice that lowerBound and upperBound are used only to decide the partition stride, not for<br>filtering the rows in the table. Thus, all rows in the table will be partitioned and returned. This option<br>applies only to reading. |  |
| numPartitions | The maximum number of partitions that can be used for parallelism in table reading and writing. This<br>also determines the maximum number of concurrent JDBC connections. If the number of partitions<br>to write exceeds this limit, we decrease it to this limit by calling coalesce(numPartitions) before<br>writing. |  |
| fetchsize | The JDBC fetch size, which determines how many rows to fetch per round trip. This can help<br>performance on JDBC drivers, which default to low fetch size (e.g., Oracle with 10 rows). This<br>option applies only to reading. |  |
| batchsize | The JDBC batch size, which determines how many rows to insert per round trip. This can help<br>performance on JDBC drivers. This option applies only to writing. The default is 1000. |  |
| isolationLevel | The transaction isolation level, which applies to current connection. It can be one of NONE,<br>READ_COMMITTED, READ_UNCOMMITTED, REPEATABLE_READ, or SERIALIZABLE, corresponding to<br>standard transaction isolation levels defined by JDBC’s Connection object. The default is<br>READ_UNCOMMITTED. This option applies only to writing. For more information, refer to the<br>documentation in java.sql.Connection. |  |
| truncate | This is a JDBC writer-related option. When SaveMode.Overwrite is enabled, Spark truncates an<br>existing table instead of dropping and re-creating it. This can be more efficient, and it prevents the<br>table metadata (e.g., indices) from being removed. However, it will not work in some cases, such as<br>when the new data has a different schema. The default is false. This option applies only to writing. |  |
| createTableOptions | This is a JDBC writer-related option. If specified, this option allows setting of database-specific table<br>and partition options when creating a table (e.g., CREATE TABLE t (name string)<br>ENGINE=InnoDB). This option applies only to writing. |  |
| createTableColumnTypes | The database column data types to use instead of the defaults, when creating the table. Data type<br>information should be specified in the same format as CREATE TABLE columns syntax (e.g.,<br>“name CHAR(64), comments VARCHAR(1024)”). The specified types should be valid Spark SQL<br>data types. This option applies only to writing. |  |

### Reading from SQL Databases


In [ ]:
# in Python
driver = "org.sqlite.JDBC"
path = "data/Spark-The-Definitive-Guide/data/flight-data/jdbc/my-sqlite.db"
url = "jdbc:sqlite:" + path
tablename = "flight_info"

In [ ]:
# in Python
dbDataFrame = sp.read.format("jdbc").option("url", url)\
.option("dbtable", tablename).option("driver", driver).load()

In [ ]:
# in Python
pgDF = sp.read.format("jdbc")\
.option("driver", "org.postgresql.Driver")\
.option("url", "jdbc:postgresql://database_server")\
.option("dbtable", "schema.tablename")\
.option("user", "username").option("password", "my-secret-password").load()

## Advanced I/O Concepts
We can also control specific data layout by controlling two things: bucketing and partitioning

### Splittable File Types and Compression
- Certain file formats are fundamentally “splittable.” This can improve speed because it makes it possible for Spark to avoid reading an entire file, and access only the parts of the file necessary to satisfy your query.
- Additionally if you’re using something like Hadoop Distributed File System (HDFS), splitting a file can provide further optimization if that file spans multiple blocks.

### Reading Data in Parallel
- Multiple executors cannot read from the same file at the same time necessarily, but they can read different files at the same time.
- In general, this means that when you read from a folder with multiple files in it, each one of those files will become a partition in your DataFrame and be read in by available executors in parallel (with the remaining queueing up behind the others).

### Writing Data in Parallel
- The number of files or data written is dependent on the number of partitions the DataFrame has at the time you write out the data.
- By default, one file is written per partition of the data.

Example:
`csvFile.repartition(5).write.format("csv").save("/tmp/multiple.csv")`

ls /tmp/multiple.csv

/tmp/multiple.csv/part-00000-767df509-ec97-4740-8e15-4e173d365a8b.csv

/tmp/multiple.csv/part-00001-767df509-ec97-4740-8e15-4e173d365a8b.csv

/tmp/multiple.csv/part-00002-767df509-ec97-4740-8e15-4e173d365a8b.csv

/tmp/multiple.csv/part-00003-767df509-ec97-4740-8e15-4e173d365a8b.csv

/tmp/multiple.csv/part-00004-767df509-ec97-4740-8e15-4e173d365a8b.csv


### Partitioning
- Partitioning is a tool that allows you to control what data is stored (and where) as you write it. When you write a file to a partitioned directory (or table), you basically encode a column as a folder. What this allows you to do is skip lots of data when you go to read it in later, allowing you to read in only the data relevant to your problem instead of having to scan the complete dataset. These are supported 

for all file-based data sources: